# Processing dump from arrests database

In [117]:
import pandas as pd
import geopandas as gpd
import requests
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import json
import altair_latimes as lat
import altair as alt
from pandas.io.json import json_normalize 
alt.renderers.enable('notebook')
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

In [123]:
lapd_arrest = pd.read_csv('input/lapd_arrest.csv', low_memory=False).fillna('').drop_duplicates()
lapd_charge = pd.read_csv('input/lapd_charge.csv', low_memory=False).fillna('').drop_duplicates()
lapd_legalcode = pd.read_csv('input/lapd_legalcode.csv', low_memory=False).fillna('').drop_duplicates()

In [128]:
len(lapd_arrest)

998342

In [129]:
len(lapd_charge)

1311025

In [130]:
len(lapd_legalcode)

3573

In [131]:
lapd_arrest.sort_values(by='id', ascending=True).head(1)

,id,attachment_id,booking_num,booking_date,jail,first_name,middle_name,last_name,suffix,gender,...,charge_string,total_bail,no_bail,release_date,release_reason,release_location,release_agency,has_been_amended,arr_div,arrest_cross_street
20,14374,55,2773458,2011-06-13 00:01:00-07,4212,MAINARD,,MARTIN,,M,...,CORPORAL INJURY ON SPOUSE/COHABITANT/ET,50000,f,,,,,f,,


In [132]:
lapd_charge.sort_values(by='id', ascending=True).head(1)

,id,arrest_id,legal_code_id,level,bail_amount,no_bail
0,19254,14374,635,F,50000,f


In [134]:
lapd_legalcode.head(1)

,id,raw_code,name,clean_code,slug
0,1,11383.7HS,,,113837hs


In [135]:
len(lapd_charge)

1311025

In [103]:
merged = pd.merge(lapd_arrest,lapd_charge,left_on='id',right_on='arrest_id')
merged.head()

,id_x,attachment_id,booking_num,booking_date,jail,first_name,middle_name,last_name,suffix,gender,...,release_agency,has_been_amended,arr_div,arrest_cross_street,id_y,arrest_id,legal_code_id,level,bail_amount,no_bail_y
0,41825,156,2660329,2011-03-02 16:20:00-08,4279,DAREON,JOSHUA,WILLIAMS,,M,...,,f,,,55785,41825,555,M,20000,f
1,128894,494,2295050,2010-04-12 11:57:00-07,4279,HECTOR,V,VICENTE,,M,...,,f,,,171322,128894,1086,M,1000,f
2,14415,55,2773646,2011-06-13 04:43:00-07,4279,RYAN,THOMAS,WILLIAMS,,M,...,,f,,,19307,14415,743,M,2525,f
3,14383,55,2773496,2011-06-13 00:58:00-07,4279,SHEFFIELD,NANYAMKAK,WILSON,,F,...,,f,,,19265,14383,527,M,15000,f
4,14406,55,2773613,2011-06-13 03:46:00-07,4212,JUSTIN,,ASENCIO,,M,...,,f,,,19296,14406,527,M,15000,f


In [136]:
homeless = merged[(merged['arrestee_address'].str.contains('1942 TRANSIENT')) &\
                   (merged['booking_date'] > '2019-01-01')]

In [137]:
len(homeless)

15655

In [138]:
homeless_charges = homeless.groupby(['charge_string']).agg('size')\
    .reset_index(name='arrests').sort_values(by='arrests',ascending=False)

In [139]:
homeless_charges.head(100)

,charge_string,arrests
0,,2069
141,ADW W/O FIREARM/GBI,410
1056,POSSESSION CONTROLLED SUBSTANCE,397
259,BATTERY ON PERSON,332
1441,ROBBERY,322
338,BURGLARY,290
1,and,246
1837,VIOLATION OF PAROLE,236
382,CORPORAL INJURY ON SPOUSE/COHABITANT/ET,203
1562,TAKE VEHICLE W/O OWNER'S CONSENT,203
